# 🤖 Rahhal AI Models (Colab Version)
This notebook demonstrates the two core AI models used in the Rahhal project:
1. **Crowd Prediction Model:** Uses Facebook Prophet for time-series forecasting.
2. **Recommendation System:** Uses Deep Learning (TensorFlow/Keras) for personalized ride suggestions.

In [ ]:
!pip install flask-ngrok pyngrok joblib pandas prophet tensorflow scikit-learn

## 1. Crowd Prediction API (Prophet)

In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import joblib
import pandas as pd

app = Flask(__name__)
run_with_ngrok(app)

# Load Crowd Model
try:
    crowd_model = joblib.load('crowd_model.pkl')
    print("✅ Crowd Model Loaded")
except:
    print("⚠️ Please upload crowd_model.pkl")

@app.route('/predict_crowd', methods=['POST'])
def predict_crowd():
    data = request.json
    features = data['features'] # [temp, humidity, is_weekend]
    
    future = pd.DataFrame({
        'ds': [pd.Timestamp.now()],
        'temperature': [features[0]],
        'humidity': [features[1]],
        'is_weekend': [features[2]]
    })
    
    forecast = crowd_model.predict(future)
    prediction = int(forecast['yhat'].values[0])
    return jsonify({'prediction': prediction})


## 2. Recommendation System (Deep Learning)

In [ ]:
import tensorflow as tf
import numpy as np

# Load DL Model
try:
    rec_model = tf.keras.models.load_model('recommendation_model.h5')
    scaler = joblib.load('scaler.pkl')
    print("✅ Deep Learning Model Loaded")
except:
    print("⚠️ Please upload recommendation_model.h5 and scaler.pkl")

@app.route('/recommend', methods=['POST'])
def recommend():
    data = request.json
    # Features: [age, weight, acc_val, pref_family, pref_thrill, pref_food]
    features = data['features'] 
    
    # Scale Input
    input_scaled = scaler.transform([features])
    
    # Predict Cluster
    pred = rec_model.predict(input_scaled)
    cluster = int(round(pred[0][0]))
    
    return jsonify({'cluster': cluster, 'message': 'Deep Learning Prediction Successful'})

if __name__ == '__main__':
    app.run()